<h1> Buenos Aires Neighborhood Battle </h1>

### Importing the libraries

In [1]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation

!pip install geopy 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

!pip install folium
import folium # plotting library

print('Folium installed')
print('Libraries imported.')

     |████████████████████████████████| 92kB 6.2MB/s eta 0:00:011
Folium installed
Libraries imported.


In [2]:
!pip install beautifulsoup4
!pip install lxml
!pip install request

  Stored in directory: /home/dsxuser/.cache/pip/wheels/30/84/5f/484cfba678967ef58c16fce6890925d5c7172622f20111fbfd
  Stored in directory: /home/dsxuser/.cache/pip/wheels/c1/e3/c1/d02c8c58538853e4c9b78cadb74f6d5c5c370b48a69a7271aa
  Stored in directory: /home/dsxuser/.cache/pip/wheels/c3/c3/24/b5c132b537ab380c02d69e6bd4dec1f5db56b5fe19030473d7
  Stored in directory: /home/dsxuser/.cache/pip/wheels/d6/a4/78/01b20a9dc224dcc009fab669f7f27b943b8889c5150bd68d8a
  Stored in directory: /home/dsxuser/.cache/pip/wheels/23/7c/6e/f5b4e09d6596c8b8802b347e48f149031e2363368048f1347a
Successfully built request get post query-string public


### Getting the data from the web page.

In [3]:
from bs4 import BeautifulSoup
#getting the source code from the url
source = requests.get('https://www.coordenadas.com.es/argentina/pueblos-de-ciudad-buenos-aires/7/1').text

In [4]:
#set the html object
soup = BeautifulSoup(source,'lxml')

In [5]:
#Extracting the table from the source code
table=soup.find('table',class_='table')

### Processing the data & Visualizing in the map.

In [6]:
#Creating a list to contain the text from the rows of the HTML table
row_test=[]
for row in table.find_all('tr'):
    row_test.append(row.text)

In [7]:
#Separate each value
row_test=[value.split(" ") for value in row_test]
row_test=row_test[1:]

In [8]:
row_test=[value[2:] for value in row_test]

In [9]:
#Saving in list all values
barrio=[value[0] for value in row_test]
coor=[value[1] for value in row_test]
lat=[value.split(',')[0] for value in coor]
lon=[value.split(',')[1] for value in coor]

In [10]:
df_coor=pd.DataFrame({'barrio' : barrio, 'latitude' : lat, 'longitude' : lon})
df_coor.shape

(86, 3)

In [11]:
address = 'Buenos Aires, AR'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Buenos Aires City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Buenos Aires City are -34.6075682, -58.4370894.


In [13]:
# create map of New York using latitude and longitude values
map_ba = folium.Map(location=[latitude, longitude], zoom_start=12)

# add markers to map
for lat, lng, neighborhood in zip(df_coor['latitude'], df_coor['longitude'], df_coor['barrio']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_ba)  
    
map_ba

#### Next, we are going to start utilizing the Foursquare API to explore the neighborhoods and segment them.

In [14]:
CLIENT_ID = 'QBNEH0A5SL5FIN0SD4MRT5NEIDQPSOTNWUWDACM0COSEIZWY' # your Foursquare ID
CLIENT_SECRET = 'GEHTZ4ZQJP3JLMCNKZDNC3DAYMYHD41L1SLM3QU10EW2GVJR' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 20
latitude = lat
longitude = lng
radius = 350

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: QBNEH0A5SL5FIN0SD4MRT5NEIDQPSOTNWUWDACM0COSEIZWY
CLIENT_SECRET:GEHTZ4ZQJP3JLMCNKZDNC3DAYMYHD41L1SLM3QU10EW2GVJR


#### Now, let's get the top 20 venues that are in the neighborhoods within a radius of 200 meters.

In [15]:
#Creating a list with the neighborhood & the url for the Foursquare API 
def getNearbyVenues(names, latitudes, longitudes, radius=200):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [16]:
barrios_venues = getNearbyVenues(names=df_coor['barrio'],latitudes=df_coor['latitude'],longitudes=df_coor['longitude'])

Agronomia
Almagro
Almirante-Brown
Balbastro
Balvanera
Barracas
Barrio-Norte
Belgrano
Boca
Boedo
Bonorino
Buenos-Aires
Caballito
Cafferata
Calaza
Centro
Chacarita
Ciudad-Autonoma-de-Buenos-Aires
Coghlan
Colegiales
Colelache
Colhue-Huapi
Comandante-L-Piedrabuena
Congreso
Constitucion
Dolavon
Dos-Pozos
El-Maiten
El-Sombrero
Emilio-Mitre
Epuyen
Escalante
Facundo
Flores
Floresta
Gan-Gan
General-Jose-de-San-Martin
Isla-Demarchi
Lacarra
Liniers
Los-Perales
Luis-J-Garcia
Marcelo-Torcuato-de-Alvear
Mataderos
Monte-Castro
Montserrat
Nazca
Nueva-Chicago
Nueva-Pompeya
Nunez
Once
Palermo
Parque-Chacabuco
Parque-Chas
Parque-Patricios
Paternal
Presidente-Rivadavia
Presidente-Roque-Saenz-Pena
Primera-Junta
Puerto-Madero
Puerto-Nuevo
Pueyrredon
Ramon-L-Falcon
Recoleta
Retiro
Saavedra
San-Cristobal
San-Telmo
Simon-Bolivar
Tellier
Varela
Velez-Sarsfield
Versailles
Villa-Crespo
Villa-del-Parque
Villa-Devoto
Villa-General-Mitre
Villa-Lamadrid
Villa-Lugano
Villa-Luro
Villa-Ortuzar
Villa-Real
Villa-Riachuelo

In [17]:
# Printing the shape & head of the df
print(barrios_venues.shape)
barrios_venues.head()

(312, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Agronomia,-34.6,-58.48333,Parada Linea 146,-34.599644,-58.481397,Bus Stop
1,Almagro,-34.6,-58.41667,Pierino,-34.600738,-58.415138,Italian Restaurant
2,Almagro,-34.6,-58.41667,Musetta Caffé,-34.599670,-58.415087,Café
3,Almagro,-34.6,-58.41667,DNI Tango,-34.599726,-58.417791,Dance Studio
4,Almagro,-34.6,-58.41667,Le Blé,-34.599125,-58.416058,Coffee Shop


### Processing the data of the venues

In [21]:
#Printing the unique categories of venues
barrios_venues['Venue Category'].unique()

array(['Bus Stop', 'Italian Restaurant', 'Coffe Shop', 'Dance Studio',
       'Coffee Shop', 'Theater', 'Gym / Fitness Center', 'Hot Dog Joint',
       'Rock Club', 'Bar', 'Tapas Restaurant', 'Pizza Place', 'Pub',
       'Lounge', 'Soccer Stadium', 'BBQ Joint', 'Electronics Store',
       'Hotel', 'Metro Station', 'Art Museum', 'Japanese Restaurant',
       'French Restaurant', 'Pharmacy', 'Seafood Restaurant',
       'Gas Station', 'Bakery', 'Plaza', 'Athletics & Sports',
       'Argentinian Restaurant', 'Brewery', 'Ice Cream Shop',
       'Intersection', 'Bistro', 'Hostel',
       'Vegetarian / Vegan Restaurant', 'Liquor Store', 'Hobby Shop',
       'Martial Arts Dojo', 'Opera House', 'Indian Restaurant',
       'Event Space', 'Photography Lab', 'Deli / Bodega',
       'Cultural Center', 'Taco Place', 'Bagel Shop', 'Restaurant',
       'Clothing Store', 'Supermarket', 'Burger Joint', 'Diner',
       'Bookstore', 'Gastropub', 'Sandwich Place', 'Cheese Shop',
       'Convenience Store'

In [19]:
# Café = Coffe Shop
barrios_venues['Venue Category'].replace('Café','Coffe Shop',inplace=True)
# Gym / Fitness Center = Gym 
barrios_venues['Venue Category'].replace('Gym','Gym / Fitness Center',inplace=True)

In [20]:
print('There are {} uniques categories.'.format(len(barrios_venues['Venue Category'].unique())))

There are 81 uniques categories.


In [22]:
# one hot encoding
barrios_onehot = pd.get_dummies(barrios_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
barrios_onehot['Neighborhood'] = barrios_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [barrios_onehot.columns[-1]] + list(barrios_onehot.columns[:-1])
barrios_onehot = barrios_onehot[fixed_columns]

barrios_onehot.head()

,Vegetarian / Vegan Restaurant,American Restaurant,Argentinian Restaurant,Art Museum,Arts & Entertainment,Athletics & Sports,BBQ Joint,Bagel Shop,Bakery,Bar,...,Shop & Service,Soccer Stadium,Sporting Goods Shop,Sports Club,Steakhouse,Supermarket,Taco Place,Tapas Restaurant,Theater,Trail
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [23]:
barrios_onehot.shape

(312, 81)

In [24]:
barrios_grouped = barrios_onehot.groupby('Neighborhood').mean().reset_index()
barrios_grouped

,Neighborhood,Vegetarian / Vegan Restaurant,American Restaurant,Argentinian Restaurant,Art Museum,Arts & Entertainment,Athletics & Sports,BBQ Joint,Bagel Shop,Bakery,...,Shop & Service,Soccer Stadium,Sporting Goods Shop,Sports Club,Steakhouse,Supermarket,Taco Place,Tapas Restaurant,Theater,Trail
0,Agronomia,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,...,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0
1,Almagro,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,...,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.071429,0.142857,0.0
2,Almirante-Brown,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,...,0.000000,1.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0
3,Balvanera,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.166667,0.0,0.000000,...,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0
4,Barrio-Norte,0.000000,0.000000,0.000000,0.076923,0.0,0.0,0.076923,0.0,0.076923,...,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0
5,Belgrano,0.000000,0.000000,0.000000,0.000000,0.0,0.5,0.000000,0.0,0.000000,...,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0
6,Boedo,0.000000,0.000000,0.400000,0.000000,0.0,0.0,0.200000,0.0,0.000000,...,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0
7,Bonorino,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.250000,...,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0
8,Buenos-Aires,0.083333,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.083333,...,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0
9,Caballito,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,...,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0


In [25]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [26]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = barrios_grouped['Neighborhood']

for ind in np.arange(barrios_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(barrios_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agronomia,Bus Stop,Theater,Cheese Shop,Clothing Store,Cocktail Bar,Coffe Shop,Coffee Shop,Convenience Store,Cultural Center,Dance Studio
1,Almagro,Theater,Rock Club,Pub,Tapas Restaurant,Lounge,Gym / Fitness Center,Italian Restaurant,Dance Studio,Hot Dog Joint,Bar
2,Almirante-Brown,Soccer Stadium,Trail,Dance Studio,Cheese Shop,Clothing Store,Cocktail Bar,Coffe Shop,Coffee Shop,Convenience Store,Cultural Center
3,Balvanera,Metro Station,Coffe Shop,Electronics Store,BBQ Joint,Hotel,Bus Stop,Trail,Cultural Center,Clothing Store,Cocktail Bar
4,Barrio-Norte,Coffe Shop,Gas Station,Art Museum,French Restaurant,Japanese Restaurant,BBQ Joint,Pharmacy,Bakery,Seafood Restaurant,Italian Restaurant


### Clustering the neighborhood with a K-Mean

In [27]:
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

# set number of clusters
kclusters = 5

barrios_grouped_clustering = barrios_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(barrios_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_

array([2, 1, 3, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 3, 4, 1, 2, 1, 3, 2, 1, 1, 3, 0, 1, 1, 1, 1, 1, 1, 2, 1, 1,
       1, 1, 0, 1, 1, 0, 1, 1, 2, 1, 1, 4, 1, 0, 1, 1, 2, 1, 1, 1, 1, 1,
       3, 1, 2], dtype=int32)

In [28]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)
df_coor['Neighborhood'] = df_coor['barrio']
df_coor.drop('barrio', 1)

barrios_merged = df_coor

# merge barrios_grouped with barrios_data to add latitude/longitude for each neighborhood
barrios_merged = barrios_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

barrios_merged.head() # check the last columns!

,barrio,latitude,longitude,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agronomia,-34.6,-58.48333,Agronomia,2.0,Bus Stop,Theater,Cheese Shop,Clothing Store,Cocktail Bar,Coffe Shop,Coffee Shop,Convenience Store,Cultural Center,Dance Studio
1,Almagro,-34.6,-58.41667,Almagro,1.0,Theater,Rock Club,Pub,Tapas Restaurant,Lounge,Gym / Fitness Center,Italian Restaurant,Dance Studio,Hot Dog Joint,Bar
2,Almirante-Brown,-34.66667,-58.45,Almirante-Brown,3.0,Soccer Stadium,Trail,Dance Studio,Cheese Shop,Clothing Store,Cocktail Bar,Coffe Shop,Coffee Shop,Convenience Store,Cultural Center
3,Balbastro,-34.65,-58.46667,Balbastro,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Balvanera,-34.61018,-58.40654,Balvanera,1.0,Metro Station,Coffe Shop,Electronics Store,BBQ Joint,Hotel,Bus Stop,Trail,Cultural Center,Clothing Store,Cocktail Bar


In [29]:
barrios_merged.dropna(inplace=True)

In [30]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=12)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(barrios_merged['latitude'], barrios_merged['longitude'], barrios_merged['Neighborhood'], barrios_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster)-1],
        fill=True,
        fill_color=rainbow[int(cluster)-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

# Example with a Coffe Shop in Caballito

In [58]:
#Getting the venue information
caballito_venue= barrios_venues[barrios_venues['Neighborhood']=='Caballito']
neig_name='Caballito'
nei_lat=caballito_venue['Neighborhood Latitude'][58]
nei_lng=caballito_venue['Neighborhood Longitude'][58]
caballito_venue

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
58,Caballito,-34.61667,-58.45,Mr. Wines,-34.616937,-58.448109,Liquor Store
59,Caballito,-34.61667,-58.45,La Veneziana,-34.615786,-58.451821,Ice Cream Shop
60,Caballito,-34.61667,-58.45,El Viejo Buzón,-34.615817,-58.449193,Coffee Shop
61,Caballito,-34.61667,-58.45,Menenga,-34.617072,-58.448546,Bistro


In [60]:
# create map of Caballito using latitude and longitude values
map_neig = folium.Map(location=[nei_lat, nei_lng], zoom_start=16)

# add markers to map
for lat, lng, venue, cat in zip(devoto_venue['Venue Latitude'], devoto_venue['Venue Longitude'], devoto_venue['Venue'], devoto_venue['Venue Category']):
    label = '{}, {}'.format(venue, cat)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_neig)  
    
map_neig

### Conclusion

**If you want to rent a space for a Coffe Shop, you have to know that in the intersection of Gervacio Espinosa and Neuquen there is a Coffe Shop called 'El Viejo Buzon'. You should set your venue some streets further there. Or you should check for similar neighborhoods like:**

In [61]:
barrios_merged[barrios_merged['Cluster Labels']==1]

,barrio,latitude,longitude,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Almagro,-34.6,-58.41667,Almagro,1.0,Theater,Rock Club,Pub,Tapas Restaurant,Lounge,Gym / Fitness Center,Italian Restaurant,Dance Studio,Hot Dog Joint,Bar
4,Balvanera,-34.61018,-58.40654,Balvanera,1.0,Metro Station,Coffe Shop,Electronics Store,BBQ Joint,Hotel,Bus Stop,Trail,Cultural Center,Clothing Store,Cocktail Bar
6,Barrio-Norte,-34.58333,-58.4,Barrio-Norte,1.0,Coffe Shop,Gas Station,Art Museum,French Restaurant,Japanese Restaurant,BBQ Joint,Pharmacy,Bakery,Seafood Restaurant,Italian Restaurant
9,Boedo,-34.63333,-58.41667,Boedo,1.0,Argentinian Restaurant,Brewery,BBQ Joint,Ice Cream Shop,Trail,Deli / Bodega,Cocktail Bar,Coffe Shop,Coffee Shop,Convenience Store
10,Bonorino,-34.63333,-58.46667,Bonorino,1.0,Lounge,Ice Cream Shop,Bakery,Intersection,Trail,Dance Studio,Clothing Store,Cocktail Bar,Coffe Shop,Coffee Shop
11,Buenos-Aires,-34.61315,-58.37723,Buenos-Aires,1.0,Hotel,Coffe Shop,Vegetarian / Vegan Restaurant,Gym / Fitness Center,Hostel,Bakery,Dance Studio,Bistro,Deli / Bodega,Cocktail Bar
12,Caballito,-34.61667,-58.45,Caballito,1.0,Coffee Shop,Liquor Store,Ice Cream Shop,Bistro,Trail,Dance Studio,Clothing Store,Cocktail Bar,Coffe Shop,Convenience Store
13,Cafferata,-34.63333,-58.43333,Cafferata,1.0,Gym / Fitness Center,Hobby Shop,Pizza Place,Martial Arts Dojo,Trail,Cultural Center,Clothing Store,Cocktail Bar,Coffe Shop,Coffee Shop
15,Centro,-34.6,-58.38333,Centro,1.0,Coffe Shop,Plaza,Coffee Shop,Opera House,Theater,Event Space,Indian Restaurant,Bistro,Cultural Center,Cheese Shop
16,Chacarita,-34.58333,-58.45,Chacarita,1.0,Vegetarian / Vegan Restaurant,Bagel Shop,Deli / Bodega,Dance Studio,Cultural Center,Brewery,Photography Lab,Bar,Theater,Taco Place
